# Prompt Engineering using Gemini and Python

In [1]:
!pip install -q -U google-generativeai


In [2]:
!pip install -q --upgrade google-generativeai


In [3]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
from google.colab import userdata

In [13]:
import os
import getpass

api_key = getpass.getpass('Gemini API Key:')

genai.configure(api_key=api_key)

Gemini API Key:··········


In [15]:
model = genai.GenerativeModel('gemini-pro')


In [16]:
%%time
response = model.generate_content("What is the meaning of life?")

CPU times: user 111 ms, sys: 8.57 ms, total: 120 ms
Wall time: 5.95 s


In [18]:
response.text

"The meaning of life is a deeply personal and subjective question that has occupied philosophers, theologians, and individuals throughout history. There is no one universally accepted answer, but there are a multitude of proposed meanings, including:\n\n1. **Eudaimonia (Happiness):** The pursuit of happiness and well-being as the ultimate goal.\n\n2. **Purpose:** Fulfilling a specific purpose or role in life, often related to one's talents or passions.\n\n3. **Service:** Making a positive contribution to society or others, finding meaning through acts of kindness and empathy.\n\n4. **Growth and Learning:** The ongoing pursuit of knowledge, skill development, and personal evolution.\n\n5. **Connection:** Building and maintaining meaningful relationships with others, finding belonging and purpose within a community.\n\n6. **Legacy:** Leaving behind a positive and lasting impact on the world, whether through accomplishments, creativity, or mentorship.\n\n7. **Spiritual Fulfillment:** Seek

In [19]:
to_markdown(response.text)

> The meaning of life is a deeply personal and subjective question that has occupied philosophers, theologians, and individuals throughout history. There is no one universally accepted answer, but there are a multitude of proposed meanings, including:
> 
> 1. **Eudaimonia (Happiness):** The pursuit of happiness and well-being as the ultimate goal.
> 
> 2. **Purpose:** Fulfilling a specific purpose or role in life, often related to one's talents or passions.
> 
> 3. **Service:** Making a positive contribution to society or others, finding meaning through acts of kindness and empathy.
> 
> 4. **Growth and Learning:** The ongoing pursuit of knowledge, skill development, and personal evolution.
> 
> 5. **Connection:** Building and maintaining meaningful relationships with others, finding belonging and purpose within a community.
> 
> 6. **Legacy:** Leaving behind a positive and lasting impact on the world, whether through accomplishments, creativity, or mentorship.
> 
> 7. **Spiritual Fulfillment:** Seeking meaning and purpose through religious beliefs, meditation, or spiritual practices.
> 
> 8. **Hedonism:** Experiencing pleasure and enjoyment as the primary goal.
> 
> 9. **Existentialism:** Embracing the inherent absurdity and meaninglessness of existence and creating meaning through individual choices and actions.
> 
> 10. **Meaning Varies:** The meaning of life is fluid and subjective, changing according to individual experiences, beliefs, and values.
> 
> Ultimately, the meaning of life is a question that each individual must answer for themselves. It is an ongoing journey of self-discovery and introspection, with no single or definitive answer.

# Automatic Grader using Gemini


In [20]:
! pip install -q -U langchain

In [21]:
!pip install -q langchain-google-genai


In [22]:
# Run this cell and paste the API key in the prompt
import os
import getpass

os.environ['GOOGLE_API_KEY'] = api_key

In [23]:
from langchain import PromptTemplate
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.chains import LLMChain


In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI
chat_model = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.7, top_p=0.85)

In [25]:
# define prompt template
prompt_template_text = """You are a high school history teacher grading homework assignments. \
Based on the homework question indicated by “**Q:**” and the correct answer indicated by “**A:**”, your task is to determine whether the student's answer is correct. \
Grading is binary; therefore, student answers can be correct or wrong. \
Simple misspellings are okay.

**Q:** {question}
**A:** {correct_answer}

**Student's Answer:** {student_answer}
"""

prompt = PromptTemplate(input_variables=["question", "correct_answer", "student_answer"], template = prompt_template_text)

In [26]:
# define chain
chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
)

In [27]:
question = "Who was the 35th president of the United States of America?"
correct_answer = "John F. Kennedy"
student_answer =  "JFK"

# run chain
chain.run({'question':question, 'correct_answer':correct_answer, 'student_answer':student_answer})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Correct'

In [29]:
# run chain in for loop
student_answer_list = ["John F. Kennedy", "JFK", "FDR", "John F. Kenedy", "John Kennedy", "Jack Kennedy", "Jacqueline Kennedy", "Robert F. Kenedy"]

for student_answer in student_answer_list:
    print(student_answer + " - " + str(chain.run({'question':question, 'correct_answer':correct_answer, 'student_answer':student_answer})))


John F. Kennedy - Correct
JFK - Correct
FDR - Wrong
John F. Kenedy - Correct
John Kennedy - Correct
Jack Kennedy - Correct
Jacqueline Kennedy - Wrong
Robert F. Kenedy - Wrong


In [30]:
from langchain.schema import BaseOutputParser

# define output parser
class GradeOutputParser(BaseOutputParser):
    """Determine whether grade was correct or wrong"""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return "wrong" not in text.lower()

In [31]:
# update chain
chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    output_parser=GradeOutputParser()
)

In [32]:
# grade student answers
for student_answer in student_answer_list:
    print(student_answer + " - " + str(chain.run({'question':question, 'correct_answer':correct_answer, 'student_answer':student_answer})))


John F. Kennedy - True
JFK - True
FDR - False
John F. Kenedy - True
John Kennedy - True
Jack Kennedy - True
Jacqueline Kennedy - False
Robert F. Kenedy - False
